In [166]:
from torch.autograd import Variable

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Neural Network Model

nn.Module 을 사용하여 Neural Network 모델을 만들고, 반드시 forward(input)함수를 만들어줘야 합니다.<br>
backward함수는 자동으로 만들어집니다.

> torch.nn 전체 모듈 통들어서.. 오직 minibatch만 제공됩니다.<br>
> 즉 single sample은 전혀 지원되지 않습니다.<br>
> 예를 들어서 4D Tensor는 다음과 같이 사용되어야 합니다.<br><br>
> <span style="color:#cc4444; font-weight:bold;">nSamples x nChannels x Height x Width</span><br><br>
> 만약 single sample을 사용하려고 한다면 **input.unsqueeze(0)** 를 사용해서 fake batch를 만들어 줍니다.

In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5).cuda()
        self.conv2 = nn.Conv2d(6, 16, 5).cuda()
        self.fc1   = nn.Linear(16 * 5 * 5, 120).cuda()  # an affine operation: y = Wx + b
        self.fc2   = nn.Linear(120, 84).cuda()
        self.fc3   = nn.Linear(84, 10).cuda()

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))  # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)  # If the size is a square you can only specify a single number
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


### Predict

GPU에서 처리한 값을 CPU로 불러와서 처리해야 하기 때문에 .cpu 함수를 사용합니다.

In [161]:
input = Variable(torch.randn(1, 1, 32, 32)).cuda()
output = net(input)

output.data.cpu().numpy()

array([[-0.04526739,  0.01011707,  0.08764283, -0.07252117,  0.08807731,
         0.00853545, -0.01395075,  0.0785292 , -0.06747445,  0.0142101 ]], dtype=float32)

### Loss function

In [162]:
target = Variable(torch.arange(0, 10)).cuda()
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

Variable containing:
 28.4200
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [163]:
print(loss.creator)  # MSELoss
print(loss.creator.previous_functions[0][0])  # Linear
print(loss.creator.previous_functions[0][0].previous_functions[0][0])  # ReLU

### Backpropagation

backpropagate the error 하기전에 반드시 .zero_grad()를 사용해서 gradients값을 0으로 만들어줘야 합니다. <br>
이렇게 하지 않고, .backward()를 사용시 gradients값들이 accumulate되게 됩니다.

In [164]:
net.zero_grad()   

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.cuda.FloatTensor of size 6 (GPU 0)]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
  0.0102
 -5.8490
  4.7722
  1.1550
 -2.7535
  0.2246
[torch.cuda.FloatTensor of size 6 (GPU 0)]



### Update the weights

가장 간단한 Stochastic Gradient Descent에 따르면 update는 다음과 같습니다.

$$ \text{weight} = \text{weight} - \text{learning rate} * \text{gradient} $$

파이썬에서 간단하게 하려면 다음과 같이 합니다.

In [165]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

하지만 SGD, Nesterov-SGD, Adam, RMSProp 같은 다양한 방법으로의 update가 존재합니다.<br>
torch.optim 모듈은 이미 다양한 optimizers들을 제공하고 있습니다.

In [167]:
optimizer = optim.SGD(net.parameters(), lr=0.01)

# In your training loop
# loop 안에서.. 
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # update를 함

### Learnable Parameters

Learnable Parameters는 parameters() 함수를 통해서 얻을 수 있습니다.

In [7]:
params = list(net.parameters())
for param in params:
    print(param.size())

print()
print(params[4])

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])

Parameter containing:
 3.6837e-02 -2.7158e-02 -2.4049e-02  ...   2.8363e-02  2.7047e-02 -2.0186e-02
 2.6292e-02  2.1365e-04  4.3966e-02  ...   1.8812e-02 -3.9648e-02 -3.7087e-02
-3.0545e-02 -2.0370e-02 -4.4853e-03  ...  -1.6517e-02 -4.8652e-02 -6.4337e-03
                ...                   ⋱                   ...                
-2.9419e-02 -2.1754e-03 -1.3552e-02  ...  -1.5166e-02  2.2346e-03  3.7931e-02
 3.8603e-02 -3.4742e-03  2.1454e-02  ...  -1.6677e-02 -4.9337e-03  9.5037e-03
 1.9295e-02  4.7208e-02  1.8468e-02  ...  -1.0812e-02  5.0891e-03  2.5843e-02
[torch.cuda.FloatTensor of size 120x400 (GPU 0)]



### Gradients값 random으로 만들기

In [113]:
output.backward(torch.randn(1, 10).cuda())
params = list(net.parameters())
print('grad', params[1].grad)
print('data', params[1].data)

grad Variable containing:
-0.0530
-0.0365
-0.0691
-0.1326
 0.0075
 0.0064
[torch.cuda.FloatTensor of size 6 (GPU 0)]

data 
-0.0331
-0.1378
 0.1596
-0.0864
 0.0523
 0.1236
[torch.cuda.FloatTensor of size 6 (GPU 0)]



### Gradients값 zero로 만들기

.zero_grad를 사용하게 되면 gradients값을 zero로 만듭니다.

In [114]:
net.zero_grad()
params = list(net.parameters())
print('grad', params[1].grad)
print('data', params[1].data)

grad Variable containing:
 0
 0
 0
 0
 0
 0
[torch.cuda.FloatTensor of size 6 (GPU 0)]

data 
-0.0331
-0.1378
 0.1596
-0.0864
 0.0523
 0.1236
[torch.cuda.FloatTensor of size 6 (GPU 0)]



In [115]:
a = params[1]
a.data


-0.0331
-0.1378
 0.1596
-0.0864
 0.0523
 0.1236
[torch.cuda.FloatTensor of size 6 (GPU 0)]